<a href="https://colab.research.google.com/github/cartertams/uiowa/blob/main/Datathon2023Fall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HEPBD_J -> 17-18 Hepatitis B: Core antibody, Surface antigen, and Hepatitis D antibody  
HEPB_S_J -> 17-18 Hepatitis B: Surface Antibody  
HEPBD_J -> 15-16 Hepatitis B: Core antibody, Surface antigen, and Hepatitis D antibody  
HEPB_S_J -> 15-16 Hepatitis B: Surface Antibody  

Core Antibody (LBXHBC) (2015-16)

Value

1	Positive	407	407
2	Negative	6386	6793
3	Indeterminate	1	6794
.	Missing	641	7435

Surface Antigen (LBDHBG) (2015-16)


1	Positive	31	31
2	Negative	375	406
3	Indeterminate	0	406
.	Missing	7029	7435



In [ ]:
library(foreign)
library(dplyr)

data1 <- read.xport('/content/HEPBD_I.XPT')
data2 <- read.xport('/content/HEPBD_J.XPT')
mydata <- rbind(data1, data2)
data1 <- read.xport('/content/HEPB_S_I.XPT')
data2 <- read.xport('/content/HEPB_S_J.XPT')
mydata2 <- rbind(data1, data2)

mydata
mydata2

newNames <- c('ID', 'CoreAntibody', 'SurfaceAntigen', 'SurfaceAntibody')
#Core Antibody -> past infection
#Surface Antigen -> has HPV
#Surface Antibody -> vaccination
dataframe <- left_join(mydata, mydata2, by = 'SEQN')
dataframe = dataframe[,-4]
names(dataframe) <- newNames
dataframe

In [ ]:
data3 <- read.xport('/content/HEPB_S_I.XPT')
head(data3)

,SEQN,LBXHBS
,<dbl>,<dbl>
1,83732,2
2,83733,2
3,83734,2
4,83735,2
5,83736,1
6,83737,2


Only test surface antigen if core antibody test is positive. Get it when they are young, then they are more likely to contract it.

Even if you are vaccinated, sometimes you don't get antibodies.

hepatitis B core antigen (anti-HBc) -> past infection  
hepatitis B surface antigen (HBsAg) -> current chronic infection  
hepatitis B surface antibody (anti-HBs) -> immunity through vaccination  

double space is newline  
NHANES var.  (17-18)
LBDHBG - Hepatitis B surface antigen  
LBXHBC - Hepatitis B core antibody  

Y= surface antigen test result  
\+  
\-  
NA
+ X=\-
+ X=NA

X= core antibody test result


Use boosted trees and use oversampling/undersampling methods before fitting models.  
Also we need to further invesitgate the missing values in the Lab data.

In [ ]:
library(xgboost)
library(FNN)

## what libary did we use in the homework for QDA/LDA/Naivebayes


#need to select all relative features
#SEQN - participant number
#SDDSRVYR - data release cycle
#RIAGENDR - Gender
#RIDAGEYR - Age in years at screening


##EASY KEEPERS


## NOW NOT SO MUCH
#RIDAGEMN - Age in months at screening - 0 to 24 mos
#RIDRETH1 - Race/Hispanic origin
#RIDRETH3 - Race/Hispanic origin w/ NH Asian
#RIDEXMON - Six month time period
#RIDEXAGM - Age in months at exam - 0 to 19 years
#DMQMILIZ - Served active duty in US Armed Forces
#DMQADFC - Served in a foreign country
#DMDBORN4 - Country of birth
#DMDCITZN - Citizenship status
#DMDYRSUS - Length of time in US
#DMDEDUC3 - Education level - Children/Youth 6-19
#DMDEDUC2 - Education level - Adults 20+
#DMDMARTL - Marital status
#RIDEXPRG - Pregnancy status at exam
#SIALANG - Language of SP Interview
#SIAPROXY - Proxy used in SP Interview?
#SIAINTRP - Interpreter used in SP Interview?
#FIALANG - Language of Family Interview
#FIAPROXY - Proxy used in Family Interview?
#FIAINTRP - Interpreter used in Family Interview?
#MIALANG - Language of MEC Interview
#MIAPROXY - Proxy used in MEC Interview?
#MIAINTRP - Interpreter used in MEC Interview?
#AIALANGA - Language of ACASI Interview
#DMDHHSIZ - Total number of people in the Household
#DMDFMSIZ - Total number of people in the Family
#DMDHHSZA - # of children 5 years or younger in HH
#DMDHHSZB - # of children 6-17 years old in HH
#DMDHHSZE - # of adults 60 years or older in HH
#DMDHRGND - HH ref person's gender
#DMDHRAGZ - HH ref person's age in years
#DMDHREDZ - HH ref person's education level
#DMDHRMAZ - HH ref person's marital status
#DMDHSEDZ - HH ref person's spouse's education level
#WTINT2YR - Full sample 2 year interview weight
#WTMEC2YR - Full sample 2 year MEC exam weight
#SDMVPSU - Masked variance pseudo-PSU
#SDMVSTRA - Masked variance pseudo-stratum
#INDHHIN2 - Annual household income
#INDFMIN2 - Annual family income
#INDFMPIR - Ratio of family income to poverty

## May need to use some sampling data and weightinh procedures?
##DONE WITH ALL DEMO DATA


## USe these from the hepatitis labs build response
#SEQN - Respondent sequence number
#LBXHBC - Hepatitis B core antibody
#LBDHBG - Hepatitis B surface antigen



#Get as much data as we can then do resampling methods, we may not have many predictors that are viable for the current task
#but we get lots of data to make up for that


#also i don't know what else we would want to include, like what labs are fast or could be widely applied,
#or what questionaire questions we could use?
#okay definitely look at the questionaire data there could be some good stuff

#COUPLE OF EASY QUESTIONAIRE ONES TO INCLUDE
#Hospital Utilization & Access to Care
#Occupation

#also consider a multi-layer screening process, like given simple demographic data we predict that these people should be intervened with
#and ask questions. It does not have to be a one size fits all approach necessarily.

#We choose these variabels because we believe they are the most likely to be answered in a healthcare setting -> cite questionaire refusal data


In [ ]:
library(foreign)
library(data.table)
library(dplyr)

#DEMO - Demographic Data
cnames <- c('SEQN', 'RIAGENDR','RIDAGEYR','RIDRETH3','DMQMILIZ','DMDBORN4','DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'INDHHIN2')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/DEMO_G.XPT", tf <- tempfile(), mode="wb")
DEMO_G3 <- foreign::read.xport(tf)
DEMO_G3 <- DEMO_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/DEMO_H.XPT", tf <- tempfile(), mode="wb")
DEMO_H3 <- foreign::read.xport(tf)
DEMO_H3 <- DEMO_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/DEMO_I.XPT", tf <- tempfile(), mode="wb")
DEMO_I3 <- foreign::read.xport(tf)
DEMO_I3 <- DEMO_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/DEMO_J.XPT", tf <- tempfile(), mode="wb")
DEMO_J3 <- foreign::read.xport(tf)
DEMO_J3 <- DEMO_J3[cnames]

DEMO <- rbind(rbind(DEMO_G3,DEMO_H3),rbind(DEMO_I3,DEMO_J3))
DEMO$DMQMILIZ[is.na(DEMO$DMQMILIZ)] <- 9
DEMO$DMDCITZN[is.na(DEMO$DMDCITZN)] <- 9
DEMO$DMDEDUC2[is.na(DEMO$DMDEDUC2)] <- 9
DEMO$DMDMARTL[is.na(DEMO$DMDMARTL)] <- 99
DEMO$INDHHIN2[is.na(DEMO$INDHHIN2)] <- 99

#HEPB - Hepatitis B: core antibody, surface antigen, and Hepatitis D antibody
cnames <- c('SEQN', 'LBXHBC', 'LBDHBG')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/HEPBD_G.XPT", tf <- tempfile(), mode="wb")
HEPBD_G3 <- foreign::read.xport(tf)
HEPBD_G3 <- HEPBD_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/HEPBD_H.XPT", tf <- tempfile(), mode="wb")
HEPBD_H3 <- foreign::read.xport(tf)
HEPBD_H3 <- HEPBD_H3[cnames]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/HEPBD_I.XPT", tf <- tempfile(), mode="wb")
HEPBD_I3 <- foreign::read.xport(tf)
HEPBD_I3 <- HEPBD_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/HEPBD_J.XPT", tf <- tempfile(), mode="wb")
HEPBD_J3 <- foreign::read.xport(tf)
HEPBD_J3 <- HEPBD_J3[cnames]

HEPBD <- rbind(rbind(HEPBD_G3,HEPBD_H3),rbind(HEPBD_I3,HEPBD_J3))
HEPBD$LBXHBC[is.na(HEPBD$LBXHBC)] <- 3
HEPBD$LBDHBG[is.na(HEPBD$LBDHBG)] <- 2



#HEPB_S - Hepatitis B: Surface Antibody
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/HEPB_S_G.XPT", tf <- tempfile(), mode="wb")
HEPB_S_G3 <- foreign::read.xport(tf)

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/HEPB_S_H.XPT", tf <- tempfile(), mode="wb")
HEPB_S_H3 <- foreign::read.xport(tf)

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/HEPB_S_I.XPT", tf <- tempfile(), mode="wb")
HEPB_S_I3 <- foreign::read.xport(tf)

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/HEPB_S_J.XPT", tf <- tempfile(), mode="wb")
HEPB_S_J3 <- foreign::read.xport(tf)

HEPB_S <- rbind(rbind(HEPB_S_G3,HEPB_S_H3),rbind(HEPB_S_I3,HEPB_S_J3))
HEPB_S$LBXHBS[is.na(HEPB_S$LBXHBS)] <- 3


#IMQ - Immunization
cnames <- c('SEQN', 'IMQ020')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/IMQ_G.XPT", tf <- tempfile(), mode="wb")
IMQ_G3 <- foreign::read.xport(tf)
IMQ_G3 <- IMQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/IMQ_H.XPT", tf <- tempfile(), mode="wb")
IMQ_H3 <- foreign::read.xport(tf)
IMQ_H3 <- IMQ_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/IMQ_I.XPT", tf <- tempfile(), mode="wb")
IMQ_I3 <- foreign::read.xport(tf)
IMQ_I3 <- IMQ_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/IMQ_J.XPT", tf <- tempfile(), mode="wb")
IMQ_J3 <- foreign::read.xport(tf)
IMQ_J3 <- IMQ_J3[cnames]

IMQ <- rbind(rbind(IMQ_G3,IMQ_H3),rbind(IMQ_I3,IMQ_J3))


#HUQ - Hospital Utilization & Access to Care
cnames <- c('SEQN','HUQ010','HUQ051','HUQ071')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/HUQ_G.XPT", tf <- tempfile(), mode="wb")
HUQ_G3 <- foreign::read.xport(tf)
colnames(HUQ_G3)[6] <- 'HUQ051'
HUQ_G3 <- HUQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/HUQ_H.XPT", tf <- tempfile(), mode="wb")
HUQ_H3 <- foreign::read.xport(tf)
HUQ_H3 <- HUQ_H3[cnames]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/HUQ_I.XPT", tf <- tempfile(), mode="wb")
HUQ_I3 <- foreign::read.xport(tf)
HUQ_I3 <- HUQ_I3[cnames]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/HUQ_J.XPT", tf <- tempfile(), mode="wb")
HUQ_J3 <- foreign::read.xport(tf)
HUQ_J3 <- HUQ_J3[cnames]

HUQ <- rbind(rbind(HUQ_G3,HUQ_H3),rbind(HUQ_I3,HUQ_J3))

#ALQ was removed

#BPQ
cnames <- c('SEQN', 'BPQ020', 'BPQ080')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/BPQ_G.XPT", tf <- tempfile(), mode="wb")
BPQ_G3 <- foreign::read.xport(tf)
BPQ_G3 <- BPQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/BPQ_H.XPT", tf <- tempfile(), mode="wb")
BPQ_H3 <- foreign::read.xport(tf)
BPQ_H3 <- BPQ_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/BPQ_I.XPT", tf <- tempfile(), mode="wb")
BPQ_I3 <- foreign::read.xport(tf)
BPQ_I3 <- BPQ_I3[cnames]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/BPQ_J.XPT", tf <- tempfile(), mode="wb")
BPQ_J3 <- foreign::read.xport(tf)
BPQ_J3 <- BPQ_J3[cnames]

BPQ <- rbind(rbind(BPQ_G3,BPQ_H3),rbind(BPQ_I3,BPQ_J3))
BPQ$BPQ020[is.na(BPQ$BPQ020)] <- 9
BPQ$BPQ080[is.na(BPQ$BPQ080)] <- 9

#CDQ was removed

#HSQ was removed

#DBQ was removed

#DUQ was removed

#HIQ
cnames <- c('SEQN', 'HIQ011')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/HIQ_G.XPT", tf <- tempfile(), mode="wb")
HIQ_G3 <- foreign::read.xport(tf)
HIQ_G3 <- HIQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/HIQ_H.XPT", tf <- tempfile(), mode="wb")
HIQ_H3 <- foreign::read.xport(tf)
HIQ_H3 <- HIQ_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/HIQ_I.XPT", tf <- tempfile(), mode="wb")
HIQ_I3 <- foreign::read.xport(tf)
HIQ_I3 <- HIQ_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/HIQ_J.XPT", tf <- tempfile(), mode="wb")
HIQ_J3 <- foreign::read.xport(tf)
HIQ_J3 <- HIQ_J3[cnames]

HIQ <- rbind(rbind(HIQ_G3,HIQ_H3),rbind(HIQ_I3,HIQ_J3))

#INQ was removed

#MCQ
cnames <- c('SEQN','MCQ010', 'MCQ080' ,'MCQ160A', 'MCQ160B', 'MCQ160C', 'MCQ160D', 'MCQ160E', 'MCQ160F', 'MCQ160G', 'MCQ160K', 'MCQ160L', 'MCQ160M', 'MCQ160N', 'MCQ220')

download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/MCQ_G.XPT", tf <- tempfile(), mode="wb")
MCQ_G3 <- foreign::read.xport(tf)
MCQ_G3 <- MCQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/MCQ_H.XPT", tf <- tempfile(), mode="wb")
MCQ_H3 <- foreign::read.xport(tf)
MCQ_H3 <- MCQ_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/MCQ_I.XPT", tf <- tempfile(), mode="wb")
MCQ_I3 <- foreign::read.xport(tf)
MCQ_I3 <- MCQ_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/MCQ_J.XPT", tf <- tempfile(), mode="wb")
MCQ_J3 <- foreign::read.xport(tf)
MCQ_J3 <- MCQ_J3[cnames]

MCQ <- rbind(rbind(MCQ_G3,MCQ_H3),rbind(MCQ_I3,MCQ_J3))
MCQ$MCQ010[is.na(MCQ$MCQ010)] <- 9
MCQ$MCQ080[is.na(MCQ$MCQ080)] <- 9
MCQ$MCQ160A[is.na(MCQ$MCQ160A)] <- 9
MCQ$MCQ160B[is.na(MCQ$MCQ160B)] <- 9
MCQ$MCQ160C[is.na(MCQ$MCQ160C)] <- 9
MCQ$MCQ160D[is.na(MCQ$MCQ160D)] <- 9
MCQ$MCQ160E[is.na(MCQ$MCQ160E)] <- 9
MCQ$MCQ160F[is.na(MCQ$MCQ160F)] <- 9
MCQ$MCQ160G[is.na(MCQ$MCQ160G)] <- 9
MCQ$MCQ160K[is.na(MCQ$MCQ160K)] <- 9
MCQ$MCQ160L[is.na(MCQ$MCQ160L)] <- 9
MCQ$MCQ160M[is.na(MCQ$MCQ160M)] <- 9
MCQ$MCQ160N[is.na(MCQ$MCQ160N)] <- 9
MCQ$MCQ220[is.na(MCQ$MCQ220)] <- 9

#PAQ was removed

#RHQ Was removed

#SLQ
cnames <- c('SEQN', 'SLD012', 'SLQ050')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/SLQ_G.XPT", tf <- tempfile(), mode="wb")
SLQ_G3 <- foreign::read.xport(tf)
colnames(SLQ_G3)[2] <- 'SLD012'
SLQ_G3 <- SLQ_G3[-4]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/SLQ_H.XPT", tf <- tempfile(), mode="wb")
SLQ_H3 <- foreign::read.xport(tf)
colnames(SLQ_H3)[2] <- 'SLD012'
SLQ_H3 <- SLQ_H3[-4]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/SLQ_I.XPT", tf <- tempfile(), mode="wb")
SLQ_I3 <- foreign::read.xport(tf)
SLQ_I3 <- SLQ_I3[cnames]


download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/SLQ_J.XPT", tf <- tempfile(), mode="wb")
SLQ_J3 <- foreign::read.xport(tf)
SLQ_J3 <- SLQ_J3[cnames]

SLQ <- rbind(rbind(SLQ_G3,SLQ_H3),rbind(SLQ_I3,SLQ_J3))

#SMQ
cnames <- c('SEQN', 'SMQ020')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/SMQ_G.XPT", tf <- tempfile(), mode="wb")
SMQ_G3 <- foreign::read.xport(tf)
SMQ_G3 <- SMQ_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/SMQ_H.XPT", tf <- tempfile(), mode="wb")
SMQ_H3 <- foreign::read.xport(tf)
SMQ_H3 <- SMQ_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/SMQ_I.XPT", tf <- tempfile(), mode="wb")
SMQ_I3 <- foreign::read.xport(tf)
SMQ_I3 <- SMQ_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/SMQ_J.XPT", tf <- tempfile(), mode="wb")
SMQ_J3 <- foreign::read.xport(tf)
SMQ_J3 <- SMQ_J3[cnames]

SMQ <- rbind(rbind(SMQ_G3,SMQ_H3),rbind(SMQ_I3,SMQ_J3))

#WHQ was removed

#BMX
cnames <- c('SEQN', 'BMXWT', 'BMXHT')
download.file("https://wwwn.cdc.gov/nchs/nhanes/2011-2012/BMX_G.XPT", tf <- tempfile(), mode="wb")
BMX_G3 <- foreign::read.xport(tf)
BMX_G3 <- BMX_G3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2013-2014/BMX_H.XPT", tf <- tempfile(), mode="wb")
BMX_H3 <- foreign::read.xport(tf)
BMX_H3 <- BMX_H3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2015-2016/BMX_I.XPT", tf <- tempfile(), mode="wb")
BMX_I3 <- foreign::read.xport(tf)
BMX_I3 <- BMX_I3[cnames]

download.file("https://wwwn.cdc.gov/nchs/nhanes/2017-2018/BMX_J.XPT", tf <- tempfile(), mode="wb")
BMX_J3 <- foreign::read.xport(tf)
BMX_J3 <- BMX_J3[cnames]

BMX <- rbind(rbind(BMX_G3,BMX_H3),rbind(BMX_I3,BMX_J3))

data <- left_join(BMX, BPQ, by = 'SEQN')
data <- left_join(data, DEMO, by = 'SEQN')
data <- left_join(data, HEPB_S, by = 'SEQN')
data <- left_join(data, HEPBD, by = 'SEQN')
data <- left_join(data, HIQ, by = 'SEQN')
data <- left_join(data, HUQ, by = 'SEQN')
data <- left_join(data, IMQ, by = 'SEQN')
data <- left_join(data, MCQ, by = 'SEQN')
data <- left_join(data, SLQ, by = 'SEQN')
data <- left_join(data, SMQ, by = 'SEQN')
data <- na.omit(data)
data <- data[data$LBXHBS !=1 ,]
data <- data[,-16]


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
install.packages("e1071")
install.packages("caret")
install.packages("ROCR")
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘gplots’




In [ ]:

attach(data)
factors <- data %>% select(-c("INDHHIN2", "DMDHHSIZ", "RIDAGEYR", "BMXWT", "BMXHT"))
for(i in 2:length(factors)) {
  factors[i] <- as.factor(factors[,i])
}






The following objects are masked _by_ .GlobalEnv:

    BPQ020, BPQ080, DMDBORN4, DMDCITZN, DMDEDUC2, DMDHHSIZ, DMDMARTL,
    DMQMILIZ, INDHHIN2, LBXHBC, RIAGENDR, RIDRETH3


The following objects are masked from data (pos = 3):

    BMXHT, BMXWT, BPQ020, BPQ080, DMDBORN4, DMDCITZN, DMDEDUC2,
    DMDHHSIZ, DMDMARTL, DMQMILIZ, HIQ011, HUQ010, HUQ051, HUQ071,
    IMQ020, INDHHIN2, LBDHBG, LBXHBC, MCQ010, MCQ080, MCQ160A, MCQ160B,
    MCQ160C, MCQ160D, MCQ160E, MCQ160F, MCQ160G, MCQ160K, MCQ160L,
    MCQ160M, MCQ160N, MCQ220, RIAGENDR, RIDAGEYR, RIDRETH3, SEQN,
    SLD012, SLQ050, SMQ020


The following objects are masked from data (pos = 4):

    BMXHT, BMXWT, BPQ020, BPQ080, DMDBORN4, DMDCITZN, DMDEDUC2,
    DMDHHSIZ, DMDMARTL, DMQMILIZ, HIQ011, HUQ010, HUQ051, HUQ071,
    IMQ020, INDHHIN2, LBDHBG, LBXHBC, MCQ010, MCQ080, MCQ160A, MCQ160B,
    MCQ160C, MCQ160D, MCQ160E, MCQ160F, MCQ160G, MCQ160K, MCQ160L,
    MCQ160M, MCQ160N, MCQ220, RIAGENDR, RIDAGEYR, RIDRETH3, SEQN,
    SLD012, SLQ05

ERROR: ignored

In [ ]:
folds = 5
foldId <- sample(1:folds, nrow(data), replace = TRUE)
accuracyMatrix = matrix(0, nrow = folds, ncol = 5)
tprMatrix = accuracyMatrix
for (cc in 1:folds) {
    cat("Fold: ", cc, "\n")

    testIdx <- which(foldId == cc)
    trainIdx <- which(foldId != cc)

    train.df <- data[trainIdx,]
    test.df <- data[testIdx,]
    # xtrain.df <- xgbData[trainIdx,]
    # xtest.df <- xgbData[testIdx,]


  logistic <- glm(LBDHBG ~ ., data = train.df, family = 'binomial')
  linear <- lda(LBDHBG ~ ., data = train.df)
  quadratic <- qda(LBDHBG ~ ., data = train.df)
  bayes <- naive_bayes(LBDHBG ~ ., data = train.df)
  #xgboost <- xgboost(data = as.matrix(xtrain.df[,-c(23,24,25)]), label = as.numeric(as.character(xtrain.df$DRK_YN)), objective = 'binary:logistic', nrounds = 10)

  pred <- predict(logistic, test.df, type = 'response')
  pred <- ifelse(pred > .5, 1, 0)
  logisticCM <- confusionMatrix(data = as.factor(pred), reference = as.factor(test.df$LBDHBG))
  accuracyMatrix[cc, 1] <- logisticCM$overall["Accuracy"]
  tprMatrix[cc, 1] <- logisticCM$byClass["Sensitivity"]

  pred <- predict(linear, test.df, type = 'response')
  linearCM <- confusionMatrix(data = as.factor(pred$class), reference = as.factor(test.df$LBDHBG))
  accuracyMatrix[cc, 2] <- linearCM$overall["Accuracy"]
  tprMatrix[cc, 2] <- linearCM$byClass["Sensitivity"]

  pred <- predict(quadratic, test.df, type = 'response')
  quadraticCM <- confusionMatrix(data = as.factor(pred$class), reference = as.factor(test.df$LBDHBG))
  accuracyMatrix[cc, 3] <- quadraticCM$overall["Accuracy"]
  tprMatrix[cc, 3] <- quadraticCM$byClass["Sensitivity"]

  pred <- predict(bayes, test.df)
  bayesCM <- confusionMatrix(data = as.factor(pred), reference = as.factor(test.df$DRK_YN))
  accuracyMatrix[cc, 4] <- bayesCM$overall["Accuracy"]
  tprMatrix[cc, 4] <- bayesCM$byClass["Sensitivity"]

  # pred <- predict(xgboost, as.matrix(xtest.df[-c(23,24,25)]), type = 'response')
  # pred <- ifelse(pred > .5, 1, 0)
  # xgboostCM <- confusionMatrix(data = as.factor(pred), reference = as.factor(xtest.df$DRK_YN))
  # accuracyMatrix[cc, 5] <- xgboostCM$overall["Accuracy"]
  # tprMatrix[cc, 5] <- xgboostCM$byClass["Sensitivity"]
}
meanAccuracyVector = rep(0, 5)
meanTPRVector = meanAccuracyVector
for (i in 1:5){
  meanAccuracyVector[i] <- mean(accuracyMatrix[,i])
  meanTPRVector[i] <- mean(tprMatrix[,i])
}

Fold:  1 


ERROR: ignored

In [ ]:
install.packages("xgboost")